# sqlite3


In [1]:
import sqlite3

with sqlite3.connect(":memory:") as con:
    c = con.cursor()
    c.execute('''CREATE TABLE sensors (date text, city text, code text, sensor_id real, temperature real)''')

    for table in c.execute("SELECT name FROM sqlite_master WHERE type = 'table'"):
        print("Table", table[0])

    c.execute("INSERT INTO sensors VALUES ('2016-11-05','Utrecht','Red',42,15.14)")
    c.execute("SELECT * FROM sensors")
    print(c.fetchone())
    con.execute("DROP TABLE sensors")

    print("# of tables", c.execute("SELECT COUNT(*) FROM sqlite_master WHERE type = 'table'").fetchone()[0])

    c.close()

Table sensors
('2016-11-05', 'Utrecht', 'Red', 42.0, 15.14)
# of tables 0


# pandas

In [2]:
import statsmodels.api as sm
from pandas.io.sql import read_sql
import sqlite3

with sqlite3.connect(":memory:") as con:
    c = con.cursor()

    data_loader = sm.datasets.sunspots.load_pandas()
    df = data_loader.data
    rows = [tuple(x) for x in df.values]

    con.execute("CREATE TABLE sunspots(year, sunactivity)")
    con.executemany("INSERT INTO sunspots(year, sunactivity) VALUES (?, ?)", rows)
    c.execute("SELECT COUNT(*) FROM sunspots")
    print(c.fetchone())
    print("Deleted", con.execute("DELETE FROM sunspots where sunactivity > 20").rowcount, "rows")

    print(read_sql("SELECT * FROM sunspots where year < 1732", con))
    con.execute("DROP TABLE sunspots")

    c.close()

(309,)
Deleted 217 rows
      year  sunactivity
0   1700.0          5.0
1   1701.0         11.0
2   1702.0         16.0
3   1707.0         20.0
4   1708.0         10.0
5   1709.0          8.0
6   1710.0          3.0
7   1711.0          0.0
8   1712.0          0.0
9   1713.0          2.0
10  1714.0         11.0
11  1723.0         11.0


# SQLAlchemy

In [5]:
from sqlalchemy import Column, ForeignKey, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy import UniqueConstraint

Base = declarative_base()
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    name = Column(String(14), nullable=False, unique=True)

    def __repr__(self):
        return "Id=%d name=%s" %(self.id, self.name)

class Sensor(Base):
    __tablename__ = 'sensor'
    id = Column(Integer, primary_key=True)
    last = Column(Integer)
    multiplier = Column(Float)
    station_id = Column(Integer, ForeignKey('station.id'))
    station = relationship(Station)

    def __repr__(self):
        return "Id=%d last=%d multiplier=%.1f station_id=%d" 
# %(self.id, self.last, self.multiplier, self.station_id)

if __name__ == "__main__":
    print("This script is used by code further down in this notebook.")

This script is used by code further down in this notebook.


In [10]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

#from alchemy_entities import Base, Sensor, Station

def populate(engine):
    Base.metadata.bind = engine

    DBSession = sessionmaker(bind=engine)
    session = DBSession()

    de_bilt = Station(name='De Bilt')
    session.add(de_bilt)
    session.add(Station(name='Utrecht'))
    session.commit()
    print("Station", de_bilt)

    temp_sensor = Sensor(last=20, multiplier=.1, station=de_bilt)
    session.add(temp_sensor)
    session.commit()
    print("Sensor", temp_sensor)

if __name__ == "__main__":
    print("This script is used by code further down in this notebook")

This script is used by code further down in this notebook


In [12]:
#from alchemy_entities import Base, Sensor, Station
#from populate_db import populate
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import os
from pandas.io.sql import read_sql


engine = create_engine('sqlite:///demo.db')
Base.metadata.create_all(engine)
populate(engine)
Base.metadata.bind = engine
DBSession = sessionmaker()
DBSession.bind = engine
session = DBSession()

station = session.query(Station).first()

print("Query 1", session.query(Station).all())
print("Query 2", session.query(Sensor).all())
print("Query 3", session.query(Sensor).filter(Sensor.station == station).one())
print(read_sql("SELECT * FROM station", engine.raw_connection()))

try:
    os.remove('demo.db')
    print("Deleted demo.db")
except OSError:
    pass

Station Id=1 name=De Bilt
Sensor Id=%d last=%d multiplier=%.1f station_id=%d
Query 1 [Id=1 name=De Bilt, Id=2 name=Utrecht]
Query 2 [Id=%d last=%d multiplier=%.1f station_id=%d]
Query 3 Id=%d last=%d multiplier=%.1f station_id=%d
   id     name
0   1  De Bilt
1   2  Utrecht
Deleted demo.db


# Pony ORM

# dataset

In [16]:
import dataset
from pandas.io.sql import read_sql
from pandas.io.sql import to_sql
import statsmodels.api as sm

db = dataset.connect('sqlite:///:memory:')
table = db["books"]
table.insert(dict(title="NumPy Beginner's Guide", author='Ivan Idris'))
table.insert(dict(title="NumPy Cookbook", author='Ivan Idris'))
table.insert(dict(title="Learning NumPy", author='Ivan Idris'))
print(read_sql('SELECT * FROM books', db.executable.raw_connection()) )

data_loader = sm.datasets.sunspots.load_pandas()
df = data_loader.data
#write_frame(df, "sunspots", db.executable.raw_connection()) 
df.to_sql("sunspots", db.executable.raw_connection()) 

table = db['sunspots']

for row in table.find(_limit=5):
   print(row)

print("Tables", db.tables)

   id                   title      author
0   1  NumPy Beginner's Guide  Ivan Idris
1   2          NumPy Cookbook  Ivan Idris
2   3          Learning NumPy  Ivan Idris
OrderedDict([('index', 0), ('YEAR', 1700.0), ('SUNACTIVITY', 5.0)])
OrderedDict([('index', 1), ('YEAR', 1701.0), ('SUNACTIVITY', 11.0)])
OrderedDict([('index', 2), ('YEAR', 1702.0), ('SUNACTIVITY', 16.0)])
OrderedDict([('index', 3), ('YEAR', 1703.0), ('SUNACTIVITY', 23.0)])
OrderedDict([('index', 4), ('YEAR', 1704.0), ('SUNACTIVITY', 36.0)])
Tables ['books', 'sunspots']


# mongodb

In [18]:
from pymongo import MongoClient
import statsmodels.api as sm
import json
import pandas as pd

client = MongoClient()
db = client.test_database

data_loader = sm.datasets.sunspots.load_pandas()
df = data_loader.data
rows = json.loads(df.T.to_json()).values()
db.sunspots.insert_many(rows)

cursor = db['sunspots'].find({})
df =  pd.DataFrame(list(cursor))
print(df)

db.drop_collection('sunspots')

     SUNACTIVITY    YEAR                       _id
0            5.0  1700.0  5872fd2f3f7dec035bd920f2
1           11.0  1701.0  5872fd2f3f7dec035bd920f3
2           16.0  1702.0  5872fd2f3f7dec035bd920f4
3           23.0  1703.0  5872fd2f3f7dec035bd920f5
4           36.0  1704.0  5872fd2f3f7dec035bd920f6
5           58.0  1705.0  5872fd2f3f7dec035bd920f7
6           29.0  1706.0  5872fd2f3f7dec035bd920f8
7           20.0  1707.0  5872fd2f3f7dec035bd920f9
8           10.0  1708.0  5872fd2f3f7dec035bd920fa
9            8.0  1709.0  5872fd2f3f7dec035bd920fb
10           3.0  1710.0  5872fd2f3f7dec035bd920fc
11           0.0  1711.0  5872fd2f3f7dec035bd920fd
12           0.0  1712.0  5872fd2f3f7dec035bd920fe
13           2.0  1713.0  5872fd2f3f7dec035bd920ff
14          11.0  1714.0  5872fd2f3f7dec035bd92100
15          27.0  1715.0  5872fd2f3f7dec035bd92101
16          47.0  1716.0  5872fd2f3f7dec035bd92102
17          63.0  1717.0  5872fd2f3f7dec035bd92103
18          60.0  1718.0  5872f

{'nIndexesWas': 1, 'ns': 'test_database.sunspots', 'ok': 1.0}

# Redis

In [20]:
import redis
import statsmodels.api as sm
import pandas as pd

r = redis.StrictRedis()
data_loader = sm.datasets.sunspots.load_pandas()
df = data_loader.data
data = df.T.to_json()
r.set('sunspots', data)
blob = r.get('sunspots')
print(pd.read_json(blob))

              0     1     10      100   101   102     103     104     105  \
SUNACTIVITY     5    11     3    14.5    34    45    43.1    47.5    42.2   
YEAR         1700  1701  1710  1800.0  1801  1802  1803.0  1804.0  1805.0   

                106   ...       90      91    92      93    94      95    96   \
SUNACTIVITY    28.1   ...      89.9    66.6    60    46.9    41    21.3    16   
YEAR         1806.0   ...    1790.0  1791.0  1792  1793.0  1794  1795.0  1796   

                97      98      99   
SUNACTIVITY     6.4     4.1     6.8  
YEAR         1797.0  1798.0  1799.0  

[2 rows x 309 columns]


# memcached

In [25]:
import memcache
import statsmodels.api as sm
import pandas as pd

client = memcache.Client([('127.0.0.1', 11211)])
data_loader = sm.datasets.sunspots.load_pandas()
df = data_loader.data
data = df.T.to_json()
client.set('sunspots', data, time=600)
print("Stored data to memcached, auto-expire after 600 seconds")
blob = client.get('sunspots')
print(pd.read_json(blob))

Stored data to memcached, auto-expire after 600 seconds
              0     1     10      100   101   102     103     104     105  \
SUNACTIVITY     5    11     3    14.5    34    45    43.1    47.5    42.2   
YEAR         1700  1701  1710  1800.0  1801  1802  1803.0  1804.0  1805.0   

                106   ...       90      91    92      93    94      95    96   \
SUNACTIVITY    28.1   ...      89.9    66.6    60    46.9    41    21.3    16   
YEAR         1806.0   ...    1790.0  1791.0  1792  1793.0  1794  1795.0  1796   

                97      98      99   
SUNACTIVITY     6.4     4.1     6.8  
YEAR         1797.0  1798.0  1799.0  

[2 rows x 309 columns]


# Cassandra

In [28]:
from cassandra import ConsistencyLevel
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement
import statsmodels.api as sm

cluster = Cluster()
session = cluster.connect()
session.execute("CREATE KEYSPACE IF NOT EXISTS mykeyspace WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };")
session.set_keyspace('mykeyspace')
session.execute("CREATE TABLE IF NOT EXISTS sunspots (year decimal PRIMARY KEY, sunactivity decimal);")

query = SimpleStatement(
    "INSERT INTO sunspots (year, sunactivity) VALUES (%s, %s)",
    consistency_level=ConsistencyLevel.QUORUM)

data_loader = sm.datasets.sunspots.load_pandas()
df = data_loader.data
rows = [tuple(x) for x in df.values]
for row in rows:
    session.execute(query, row)

rows=session.execute("SELECT COUNT(*) FROM sunspots")
for row in rows:
    print(row)

session.execute('DROP KEYSPACE mykeyspace') 
cluster.shutdown()

Row(count=309)
